In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

## Prophet Feature Selection, so Dropping All Lag and Rolling

In [2]:
covid_df = pd.read_csv('ny_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})
covid_df = covid_df.drop(['new_cases_1 lag', 'new_cases_2_lag', 'new_cases_3_lag', 'new_cases_4_lag', 'new_cases_7_lag', 'new_cases_30_lag', 'new_cases_50_lag', 'new_cases_150_lag', 'new_cases_350_lag', 'new_deaths_1_lag', 'new_deaths_2_lag', 'new_deaths_3_lag', 'new_deaths_4_lag', 'new_deaths_7_lag', 'new_deaths_30_lag', 'new_deaths_50_lag'], axis=1)

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size] # notice here the original test test is never used

In [3]:
train_size

671

In [4]:
X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
y_train = covid_train['y']

In [5]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_1 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_1[1].to_list())

['new_cases_10_mean', 'new_cases_10_max', 'new_cases_10_min', 'new_hospitalized_patients', 'minimum_temperature_celsius', 'new_cases_10_std', 'booster_rate', 'Booster_Doses_50Plus', 'PEOPLE_DEATH_NEW_COUNT', 'relative_humidity', 'new_cases_50_std', 'dew_point', 'new_cases_100_std', 'new_deaths_10_mean', 'new_cases_50_min', 'Administered_Dose1_Recip', 'Series_Complete_5Plus', 'vaccination_rate_complete', 'vaccination_rate_1dose', 'maximum_temperature_celsius', 'Booster_Doses_18Plus', 'new_cases_50_max', 'average_temperature_celsius', 'Administered_Dose1_Recip_65Plus', 'Booster_Doses', 'Series_Complete_65Plus', 'Series_Complete_12Plus', 'new_cases_100_min', 'new_deaths_10_std', 'new_cases_50_mean']


In [6]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_2 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_2[1].to_list())

['new_cases_10_std', 'new_cases_10_mean', 'new_hospitalized_patients', 'new_cases_50_mean', 'relative_humidity', 'new_cases_10_max', 'rainfall_mm', 'dew_point', 'new_cases_100_std', 'maximum_temperature_celsius', 'new_deaths_10_std', 'new_cases_10_min', 'new_cases_50_std', 'current_intensive_care_patients', 'PEOPLE_DEATH_NEW_COUNT', 'minimum_temperature_celsius', 'average_temperature_celsius', 'new_cases_100_mean', 'new_deaths_10_mean', 'new_deaths_100_mean', 'new_deaths_100_std', 'new_deaths_50_std', 'new_deaths_10_min', 'Booster_Doses', 'new_deaths_50_mean', 'Booster_Doses_50Plus', 'new_cases_50_max', 'Administered_Dose1_Recip', 'Administered_Dose1_Recip_5Plus', 'new_deaths_50_min']


In [7]:
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier

model_c = GradientBoostingRegressor().fit(X_train, y_train)
predictors_3 = pd.concat([pd.Series(model_c.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_3[1].to_list())

['new_cases_10_max', 'new_cases_10_mean', 'new_cases_50_std', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_min', 'relative_humidity', 'new_cases_10_std', 'minimum_temperature_celsius', 'new_hospitalized_patients', 'average_temperature_celsius', 'dew_point', 'new_deaths_100_mean', 'new_deaths_10_mean', 'new_cases_100_mean', 'new_deaths_10_std', 'new_cases_100_std', 'new_deaths_100_max', 'maximum_temperature_celsius', 'new_deaths_50_mean', 'Booster_Doses', 'new_cases_50_min', 'new_deaths_50_std', 'vaccination_rate_complete', 'new_cases_50_mean', 'Administered_Dose1_Recip_12Plus', 'current_intensive_care_patients', 'Administered_Dose1_Recip_5Plus', 'Series_Complete_5Plus', 'Booster_Doses_50Plus', 'new_cases_50_max']


In [8]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier

model_d = AdaBoostRegressor(random_state=1, base_estimator = DecisionTreeRegressor(random_state=1)).fit(X_train, y_train)
predictors_4 = pd.concat([pd.Series(model_d.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_4[1].to_list())

['new_cases_10_mean', 'new_cases_50_std', 'new_cases_10_min', 'new_cases_10_max', 'relative_humidity', 'minimum_temperature_celsius', 'new_cases_50_max', 'PEOPLE_DEATH_NEW_COUNT', 'Series_Complete_Yes', 'new_hospitalized_patients', 'dew_point', 'Administered_Dose1_Recip_65Plus', 'Series_Complete_12Plus', 'new_cases_10_std', 'average_temperature_celsius', 'Booster_Doses_18Plus', 'Series_Complete_5Plus', 'Series_Complete_65Plus', 'Booster_Doses', 'new_deaths_10_mean', 'maximum_temperature_celsius', 'new_deaths_10_std', 'rainfall_mm', 'new_cases_100_min', 'booster_rate', 'new_deaths_50_std', 'new_cases_50_min', 'new_cases_100_mean', 'Booster_Doses_65Plus', 'Administered_Dose1_Recip_12Plus']


In [9]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier

model_e = RandomForestRegressor().fit(X_train, y_train)
predictors_5 = pd.concat([pd.Series(model_e.feature_importances_), pd.Series(X_train.columns)], axis=1).sort_values(by=0, ascending = False).head(30)
print(predictors_5[1].to_list())

['new_cases_10_mean', 'new_cases_10_max', 'new_cases_50_std', 'new_cases_10_min', 'Series_Complete_65Plus', 'vaccination_rate_1dose', 'new_cases_100_std', 'relative_humidity', 'vaccination_rate_complete', 'Series_Complete_5Plus', 'Series_Complete_12Plus', 'PEOPLE_DEATH_NEW_COUNT', 'Administered_Dose1_Recip_65Plus', 'new_cases_10_std', 'new_deaths_10_std', 'Administered_Dose1_Recip_12Plus', 'Series_Complete_Yes', 'new_cases_50_max', 'Booster_Doses', 'minimum_temperature_celsius', 'Booster_Doses_18Plus', 'booster_rate', 'new_cases_100_max', 'dew_point', 'maximum_temperature_celsius', 'rainfall_mm', 'Series_Complete_18Plus', 'new_deaths_10_mean', 'new_hospitalized_patients', 'Administered_Dose1_Recip_18Plus']


In [10]:
# using MARS summary for feature selection
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss') # note, terms in brackets are the hyperparameters 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
----------------------------------------------------------------
Basis Function                             Pruned  Coefficient  
----------------------------------------------------------------
(Intercept)                                No      -61576.2     
h(new_cases_10_mean-9816.9)                Yes     None         
h(9816.9-new_cases_10_mean)                No      5.40089      
h(new_cases_50_mean-7090.86)               Yes     None         
h(7090.86-new_cases_50_mean)               No      -34.5068     
h(new_deaths_10_mean-94.4)                 No      -8.40888     
h(94.4-new_deaths_10_mean)                 Yes     None         
h(new_cases_10_std-2027.37)                No      -9.19775     
h(2027.37-new_cases_10_std)                No      -0.853886    
h(new_cases_50_max-12548)                  Yes     None         
h(12548-new_cases_50_max)                  No      -0.170552    
h(new_cases_50_mean-7869.86)               No      -74.4418     
h(7869.86-new

In [11]:
def common_member(a, b, c, d, e):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
    set_4 = set(d)
    set_5 = set(e)
 
    if (set_1 & set_2 & set_3 & set_4 & set_5):
        print(set_1 & set_2 & set_3 & set_4 & set_5)


In [12]:
a = predictors_1[1].to_list()
b = predictors_2[1].to_list()
c = predictors_3[1].to_list()
d = predictors_4[1].to_list()
e = predictors_5[1].to_list()

In [13]:
full = common_member(a, b, c, d, e)
full

{'new_cases_10_std', 'new_cases_10_mean', 'new_deaths_10_std', 'PEOPLE_DEATH_NEW_COUNT', 'new_deaths_10_mean', 'Booster_Doses', 'new_cases_50_max', 'new_cases_50_std', 'dew_point', 'new_hospitalized_patients', 'minimum_temperature_celsius', 'relative_humidity', 'new_cases_10_min', 'maximum_temperature_celsius', 'new_cases_10_max'}


## XGBoost Feature Selection, includes lag and rolling

In [ ]:
import os
os._exit(00)

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
covid_df = pd.read_csv('ny_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size]

X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
X_train = X_train.astype('float32')
y_train = covid_train['y']

In [3]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_7 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(40)
print(predictors_7[1].to_list())

['new_cases_2_lag', 'new_cases_10_min', 'new_cases_10_max', 'new_cases_7_lag', 'new_cases_10_std', 'new_cases_10_mean', 'new_cases_1 lag', 'new_cases_3_lag', 'new_cases_50_std', 'minimum_temperature_celsius', 'PEOPLE_DEATH_NEW_COUNT', 'booster_rate', 'new_hospitalized_patients', 'Administered_Dose1_Recip', 'Booster_Doses_50Plus', 'Administered_Dose1_Recip_5Plus', 'vaccination_rate_complete', 'Series_Complete_12Plus', 'Series_Complete_18Plus', 'new_cases_4_lag', 'average_temperature_celsius', 'Administered_Dose1_Recip_12Plus', 'relative_humidity', 'new_deaths_50_lag', 'new_cases_100_std', 'new_deaths_3_lag', 'new_cases_350_lag', 'new_cases_50_max', 'new_cases_100_max', 'Administered_Dose1_Recip_65Plus', 'new_cases_100_min', 'new_deaths_2_lag', 'new_cases_50_lag', 'new_deaths_30_lag', 'new_deaths_1_lag', 'new_cases_30_lag', 'restrictions_on_internal_movement', 'dew_point', 'Booster_Doses', 'vaccination_rate_1dose']


In [4]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_8 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_8[1].to_list())

['new_cases_1 lag', 'new_cases_7_lag', 'new_cases_10_std', 'new_cases_4_lag', 'new_cases_3_lag', 'new_cases_30_lag', 'new_cases_2_lag', 'new_cases_10_mean', 'new_cases_10_max', 'new_hospitalized_patients', 'new_cases_150_lag', 'new_cases_350_lag', 'rainfall_mm', 'new_cases_50_mean', 'maximum_temperature_celsius', 'relative_humidity', 'new_cases_100_std', 'dew_point', 'new_deaths_7_lag', 'new_cases_10_min', 'current_intensive_care_patients', 'minimum_temperature_celsius', 'new_deaths_10_std', 'new_deaths_1_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_50_lag', 'new_deaths_50_lag', 'new_deaths_4_lag', 'new_deaths_50_std', 'new_deaths_2_lag']


In [5]:
model_g = xgb.XGBRegressor().fit(X_train, y_train, verbose=False)
predictors_6 = pd.concat([pd.Series(model_g.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_6[1].to_list())

['new_cases_10_max', 'new_cases_10_mean', 'new_cases_1 lag', 'new_cases_7_lag', 'Administered_Dose1_Recip', 'new_cases_100_mean', 'minimum_temperature_celsius', 'new_cases_350_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_2_lag', 'Booster_Doses_50Plus', 'new_cases_10_std', 'new_cases_150_lag', 'new_cases_3_lag', 'new_deaths_50_lag', 'new_cases_50_mean', 'new_deaths_10_mean', 'new_cases_4_lag', 'new_cases_10_min', 'new_deaths_10_std', 'current_intensive_care_patients', 'new_deaths_4_lag', 'new_cases_50_lag', 'rainfall_mm', 'new_deaths_1_lag', 'new_deaths_7_lag', 'new_deaths_10_min', 'new_hospitalized_patients', 'new_cases_30_lag', 'average_temperature_celsius']


In [6]:
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss', allow_missing=True) # note, terms in brackets are the hyperparameters 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
---------------------------------------------------------------------------
Basis Function                                        Pruned  Coefficient  
---------------------------------------------------------------------------
(Intercept)                                           No      15513        
h(new_cases_10_mean-9816.9)                           No      7.80703      
h(9816.9-new_cases_10_mean)                           No      -1.99817     
present(new_cases_1 lag)                              Yes     None         
missing(new_cases_1 lag)                              Yes     None         
h(new_cases_1 lag-10319)*present(new_cases_1 lag)     No      -1.3817      
h(10319-new_cases_1 lag)*present(new_cases_1 lag)     No      -0.0953862   
present(new_cases_2_lag)                              Yes     None         
missing(new_cases_2_lag)                              Yes     None         
h(new_cases_2_lag-10074)*present(new_cases_2_lag)     No      -1.50326     


In [7]:
def common_member(a, b, c):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
 
    if (set_1 & set_2 & set_3):
        print(set_1 & set_2 & set_3)


In [8]:
a = predictors_7[1].to_list()
b = predictors_8[1].to_list()
c = predictors_6[1].to_list()

In [9]:
full = common_member(a, b, c)
full

{'new_cases_350_lag', 'minimum_temperature_celsius', 'new_cases_1 lag', 'new_cases_10_max', 'new_cases_30_lag', 'new_deaths_1_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_3_lag', 'new_cases_7_lag', 'new_deaths_50_lag', 'new_cases_10_min', 'new_cases_10_mean', 'new_cases_2_lag', 'new_cases_4_lag', 'new_hospitalized_patients', 'new_cases_50_lag', 'new_cases_10_std'}
